In [1]:
import numpy as np
from urllib.request import urlopen
import scipy.optimize
import random
from math import exp
from math import log

In [2]:
def parseData(fname):
  for l in urlopen(fname):
    yield eval(l)



In [3]:
print("Reading data...")
data = list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))
print("done")

Reading data...
done


In [4]:
def feature(datum):
  feat = [1, datum['review/taste'], datum['review/appearance'], datum['review/aroma'], datum['review/palate'], datum['review/overall']]
  return feat

In [5]:
X = [feature(d) for d in data]
y = [d['beer/ABV'] >= 6.5 for d in data]

In [6]:
def inner(x,y):
  return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
  return 1.0 / (1 + exp(-x))

 


### Logistic regression by gradient ascent  


#### NEGATIVE Log-likelihood

In [7]:
def f(theta, X, y, lam):
  loglikelihood = 0
  for i in range(len(X)): 
    logit = inner(X[i], theta)
    loglikelihood -= log(1 + exp(-logit))
    if not y[i]:
      loglikelihood -= logit
  for k in range(len(theta)):
    loglikelihood -= lam * theta[k]*theta[k]
  # for debugging
  # print("ll =" + str(loglikelihood))
  return -loglikelihood 

#### NEGATIVE Derivative of log-likelihood

In [8]:
def fprime(theta, X, y, lam):
  dl = [0]*len(theta)
  for i in range(len(X)):
    logit = inner(X[i], theta)
    for k in range(len(theta)):
      dl[k] += X[i][k] * (1 - sigmoid(logit))
      if not y[i]:
        dl[k] -= X[i][k]
  for k in range(len(theta)):
    dl[k] -= lam*2*theta[k]
  return np.array([-x for x in dl])

In [9]:
rand_data = random.sample(data, int(len(data)))

In [10]:
X = [feature(d) for d in rand_data]
y = [d['beer/ABV'] >= 6.5 for d in rand_data]

In [11]:
n_train = int(len(rand_data) * (1/3))

X_train = X[:n_train]
y_train = y[:n_train]



In [12]:
n_test = int((len(rand_data) - n_train) * (1/2))

X_test = X[n_train:(n_train+n_test)]
y_test = y[n_train:(n_train+n_test)]


In [13]:
X_val = X[n_train+n_test:]
y_val = y[n_train+n_test:]


#### Train

In [14]:
def train(lam):
  theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, pgtol = 10, args = (X_train, y_train, lam))
  return theta


#### Predict 

In [15]:
def performance(theta, X, y):
    scores = [inner(theta,x) for x in X]
    predictions = [s > 0 for s in scores]
    correct = [(a==b) for (a,b) in zip(predictions,y)]
    acc = sum(correct) * 1.0 / len(correct)
    return acc


#### Validation pipeline 

In [16]:
lam = 1.0

theta = train(lam)


In [17]:
val_acc = performance(theta, X_val, y_val)
test_acc = performance(theta, X_test, y_test)

In [18]:
print("lambda = " + str(lam) + ":\tvalidation accuracy = " + str(val_acc))
print("lambda = " + str(lam) + ":\ttest accuracy = " + str(test_acc))

lambda = 1.0:	validation accuracy = 0.7169856602867942
lambda = 1.0:	test accuracy = 0.7217255654886903


### Q2

In [19]:
def performance(theta, X, y):
    scores = [inner(theta,x) for x in X]
    predictions = [s > 0 for s in scores]
    
    true_pos = [(1 if(a==b and a==1) else 0) for (a,b) in zip(predictions, y)]
    true_pos = sum(true_pos)
    
    true_neg = [(1 if(a==b and a==0) else 0) for (a,b) in zip(predictions, y)]
    true_neg = sum(true_neg)
    
    false_pos = [(1 if(a!=b and a==1) else 0) for (a,b) in zip(predictions, y)]
    false_pos = sum(false_pos)
    
    false_neg = [(1 if(a!=b and a==0) else 0) for (a,b) in zip(predictions, y)]
    false_neg = sum(false_neg)
    
    true_pos_rate = float(true_pos)/float((true_pos+false_neg))
    true_neg_rate = float(true_neg)/float((true_neg+false_pos))
    BER = 1 - 0.5*(true_pos_rate + true_neg_rate)
    
    return true_pos, true_neg, false_pos, false_neg, BER

In [20]:
true_pos, true_neg, false_pos, false_neg, BER = performance(theta, X_test, y_test)
print("lambda = " + str(lam) + ":\t# of true postives = " + str(true_pos))
print("lambda = " + str(lam) + ":\t# of true negative = " + str(true_neg))
print("lambda = " + str(lam) + ":\t# of false postives = " + str(false_pos))
print("lambda = " + str(lam) + ":\t# of false negatives = " + str(false_neg))
print("lambda = " + str(lam) + ":\tBalanced Error Rate = " + str(BER))

lambda = 1.0:	# of true postives = 9113
lambda = 1.0:	# of true negative = 2916
lambda = 1.0:	# of false postives = 3319
lambda = 1.0:	# of false negatives = 1319
lambda = 1.0:	Balanced Error Rate = 0.32937772279237043


#### Q3 

Assigning more importance to FP means that the predictor will predict FP at higher accuracy therefore reduce the number of FP than before.
To achieve this, I need to lower the possibility of predicting positive.

In [21]:
def sigmoid1(x):
  return 1.0 / (1 + exp(-x))

In [27]:
def f1(theta, X, y, lam):
  loglikelihood = 0
  for i in range(len(X)):
    logit = inner(X[i], theta)
    loglikelihood -= log(1 + exp(-logit))
    if not y[i]:
      loglikelihood -= logit
  for k in range(len(theta)):
    loglikelihood -= lam * theta[k]*theta[k]
  # for debugging
  # print("ll =" + str(loglikelihood))
  return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime1(theta, X, y, lam):
  dl = [0]*len(theta)
  for i in range(len(X)):
    logit = inner(X[i], theta)
    for k in range(len(theta)):
      dl[k] += X[i][k] * (1 - sigmoid1(logit))
      if not y[i]:
        dl[k] -= X[i][k]
  for k in range(len(theta)):
    dl[k] -= lam*2*theta[k]
  return np.array([-x for x in dl])

In [28]:
def train1(lam):
  theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f1, [0]*len(X[0]), fprime1, pgtol = 10, args = (X_train, y_train, lam))
  return theta
lam = 1
theta = train1(lam)

In [29]:
true_pos, true_neg, false_pos, false_neg, BER = performance(theta, X_test, y_test)
false_pos

3319

### Q4

In [25]:
ber = {}
for lam in [0, 0.01, 0.1, 1, 100]:
    theta = train(lam)
    val_ber = performance(theta, X_val, y_val)[4]
    ber[val_ber] = lam
    print("lambda = " + str(lam) + ":\tvalidation balanced error rate = " + str(val_ber))

best_ber = min(ber.keys())
best_lam = ber[best_ber]
theta = train(best_lam)
train_ber = performance(theta, X_train, y_train)[4]
val_ber = performance(theta, X_val, y_val)[4]
test_ber = performance(theta, X_test, y_test)[4]
print()
print("best value of lambda = " + str(lam) + ":\ttrain balanced error rate = " + str(train_ber))
print("best value of lambda = " + str(lam) + ":\tvalidation balanced error rate = " + str(val_ber))
print("best value of lambda = " + str(lam) + ":\ttest balanced error rate = " + str(test_ber))

lambda = 0:	validation balanced error rate = 0.3321638727733528
lambda = 0.01:	validation balanced error rate = 0.33202129345905806
lambda = 0.1:	validation balanced error rate = 0.3320564381370883
lambda = 1:	validation balanced error rate = 0.3325546340691665
lambda = 100:	validation balanced error rate = 0.4209951010541717

best value of lambda = 100:	train balanced error rate = 0.32989111313036745
best value of lambda = 100:	validation balanced error rate = 0.33202129345905806
best value of lambda = 100:	test balanced error rate = 0.3292136172827055
